# Objetivo

Hasta ahora existen dos contenedores uno para la API con FASTAPI, y el otro con MONGO para la BD; pero como entidades separadas. Necesitamos integrarlos de tal forma que se pueden comunicar entre si.

## PASO 1 Que los contenedores esten en la misma red

Es muy facil crear una red con docker, si no existe se crea una, si es posible con la opcion `--attachable`:

`docker network create --attachable NombreRed`

Para mi caso ya hay una red llamada `platzinet`. Asi que agregamos los contenedores a la red, agregando el argumento `--network`

```sh
docker run --detach --name tuTiendaDB --mount src=dbdata,dst=/data/db --publish 27017:27017 --network platzinet mongo:5.0
```

```sh
docker run --rm -it --publish 8000:8000  --name app-temporal --network platzinet fastapi-app-image
```

Usamos el comando `docker network inspect platzinet` y el observamos que estan en la misma red:

![image](https://imgur.com/4EEb36j.png)

Igualmente podemos usar el comando `ping` desde una shell:

`ping tuTiendaDB`

## PASO 2 

Agregar nuevas dependencias al *requirements.txt*: `motor` `pymongo`

## PASO 3

Modificar el `main.py`:

```py
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
from motor.motor_asyncio import AsyncIOMotorClient

# Create a FastAPI application instance
app = FastAPI()


# --- Configuración de MongoDB ---
# IMPORTANTE: Usa el nombre del contenedor de MongoDB dentro de la red Docker.
# Docker sabe cómo encontrar 'tuTiendaDB' dentro de la red 'platzinet'.
MONGO_DETAILS = "mongodb://tuTiendaDB:27017/"



# Define your first API endpoint (route)
@app.get("/", tags=['root'])
async def read_root_endpoint():
    nombre = 'Gustavo'
    html_content = f'<h1>Hello {nombre} From API, this is ROOT</h1>'
    return HTMLResponse(html_content)


# Define your first API endpoint (route)
@app.get("/cat", tags=['cat'])
async def read_cat_endpoint():
    nombre = 'Cat'
    html_content = f'<h1>Hello {nombre} From API, this is CAT(Antonia) Endpoint</h1>'
    return HTMLResponse(html_content)



# --- Nuevo Endpoint: ¡La verificación de conexión a MongoDB! ---
@app.get("/check-db-connection", tags=['database'])
async def check_db_connection():
    """
    Endpoint para verificar si la aplicación puede conectarse a MongoDB.
    Intenta crear un cliente y realizar una operación simple.
    """
    try:
        # Crea un cliente de MongoDB.
        # Aquí es donde intentamos conectar.
        client = AsyncIOMotorClient(MONGO_DETAILS)

        # Intenta una operación muy simple para confirmar que la conexión funciona.
        # 'ping' es un comando ligero que solo verifica si el servidor está vivo.
        await client.admin.command('ping')

        # Si llegamos aquí, la conexión fue exitosa.
        client.close() # Cierra la conexión (importante en este enfoque simple)
        return JSONResponse(content={"status": "success", 
                                     "message": "¡Conexión a MongoDB exitosa!"})

    except Exception as e:
        # Si algo falla (MongoDB no está corriendo, nombre incorrecto, etc.)
        # capturamos el error y devolvemos un mensaje de fallo.
        print(f"Error durante la verificación de conexión a MongoDB: {e}")
        raise HTTPException(status_code=500, detail=f"Fallo en la conexión a MongoDB: {e}. Asegúrate de que 'tuTiendaDB' esté corriendo y en la misma red.")
```

- agregar el *conection string* o *dadena de conexion*: `MONGO_DETAILS = "mongodb://tuTiendaDB:27017/"`. Observa que `tuTiendaDB` es el nombre del contenedor. 
- agregar un nuevo endpoint `/check-db-connection`

- `return JSONResponse(content={"status": "success", "message": "¡Conexión a MongoDB exitosa!"}) ` Convierte el diccionario de Python que le pasas ({"status": "success", "message": "..."}) a un formato JSON

**Importante**: Esta no es la forma más eficiente o recomendada para una aplicación real (ya que abriría y cerraría una conexión en cada petición), pero es la más fácil de entender para verificar la conexión. 


### Paso 4

Volver a buildiar la imagen, volver a levantar los servicios:

- `docker build --tag fastapi-app-image .`


### Paso 5 Probar

![image](https://imgur.com/mtDYNDa.png)

Como tambien si hay un fallo

![image](https://imgur.com/undefined.png)

docker run --detach --name tuTiendaDB --mount src=dbdata,dst=/data/db --publish 27017:27017 mongo:5.0


docker rm -f tuTiendaDB